In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importing Libraries

In [ ]:
import matplotlib.pyplot as plt
import PIL
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

## Processing Datasets

In [ ]:
# pixels range from 0 to 255 so dividing by 255 so value lie between 0 and 1
train_datagen = ImageDataGenerator(rescale=1./255)
train = train_datagen.flow_from_directory(
    directory = "../input/capital-alphabets-28x28/train",
    target_size=(300, 300))

In [ ]:
val = train_datagen.flow_from_directory(
    directory = "../input/capital-alphabets-28x28/validation",
    target_size=(300, 300))

In [ ]:
test = val = train_datagen.flow_from_directory(
    directory = "../input/capital-alphabets-28x28/test",
    target_size=(300, 300))

## Model Creation

In [ ]:
model = tf.keras.models.Sequential([
    #input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # 26 output neuron.
    tf.keras.layers.Dense(26, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
# Compiling model 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# extracting values from val test generator
X_val,y_val = next(val)

In [ ]:
checkpointer = EarlyStopping(monitor = 'val_accuracy', verbose = 1, restore_best_weights=True,mode="max",patience = 25)
# checkpointer to monitor accuracy and preventing overfit

In [ ]:
history = model.fit(
            train ,
            steps_per_epoch =len(train)//32, #batch size is 32
            epochs=1000,
            verbose=1,
            validation_data=(X_val,y_val),
            callbacks = [checkpointer])

## Model's Performance

In [ ]:
training_loss = history.history['loss']
val_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, val_loss, 'b-')
plt.legend(['Training Loss', 'Val Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
training_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Create count of the number of epochs
epoch_count = range(1, len(training_accuracy) + 1)

# Visualize loss history

plt.plot(epoch_count, training_accuracy, 'r--')
plt.plot(epoch_count, val_accuracy, 'b-')
plt.legend(['Training Accuracy', 'Val Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
# saving the model
model.save("alpharecognition.h5")

## Test Accuracy

In [ ]:
print(f"Test accuracy = {model.evaluate(test)[1]}")

## Test Predictions Visualization

In [ ]:
x_test,y_test = next(test)
predict = model.predict(x_test)

In [ ]:
figure = plt.figure(figsize=(20, 8))
for i, index in enumerate(np.random.choice(x_test.shape[0], size=15, replace=False)):
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(x_test[index]))
    predict_index = np.argmax(predict[index])
    true_index = np.argmax(y_test[index])
    ax.set_title("{} ({})".format(chr(predict_index+65), 
                                  chr(true_index+65)),
                                  color=("green" if predict_index == true_index else "red"))

> ###  If you find this notebook helpful, do consider to upvote.